In [ ]:
from datetime import datetime

import pandas as pd
import requests
from build.passwords import password, user
from grade_reports.grade_reports import filter_assignments, get_student_grades
from requests.auth import HTTPBasicAuth

from pykubegrader.graders.late_assignments import calculate_late_submission

api_base_url = "https://engr-131-api.eastus.cloudapp.azure.com/" 

In [ ]:
from telemetry import get_assignments_submissions


In [ ]:
self.assignments, self.student_subs = get_assignments_submissions()

In [ ]:
from grade_reports.grade_reports import filter_assignments, get_student_grades

get_student_grades

In [ ]:
from datetime import datetime

import pandas as pd
import requests
from build.passwords import password, user
from grade_reports.grade_reports import filter_assignments, get_student_grades
from requests.auth import HTTPBasicAuth

from pykubegrader.graders.late_assignments import calculate_late_submission

api_base_url = "https://engr-131-api.eastus.cloudapp.azure.com/"


# def get_all_students():
#     res = requests.get(
#         url=api_base_url.rstrip("/") + "/get-all-submission-emails",
#         auth=HTTPBasicAuth(user(), password()),
#     )

#     return res.json()


# def get_student_grades(student_id):
#     params = {"username": student_id}
#     res = requests.get(
#         url=api_base_url.rstrip("/") + "/student-grades-testing",
#         params=params,
#         auth=HTTPBasicAuth(user(), password()),
#     )

#     [assignments, sub] = res.json()

#     return assignments, sub


# def get_student(student):
#     print(student)
#     # Get assignments and submissions for the student (assumed functions)
#     assignments, submissions = get_student_grades(student)

#     # Recalculate grades and get a grades dictionary
#     grades_dict = recalculate_best_grades(assignments, submissions)

#     # Calculate averages and build a row for the student
#     row = calculate_averages(grades_dict, student)

#     # Convert the row (a dictionary) into a DataFrame
#     # row_df = pd.DataFrame([row])

#     return row


# def get_all_student_grades():
#     # Initialize an empty DataFrame to hold all student grades
#     df = pd.DataFrame()

#     # Get all students (assuming get_all_students() is a defined function)
#     students = get_all_students()

#     for student in students:
#         row_df = get_student(student)

#         # Append the row to the DataFrame
#         df = pd.concat([df, row_df], ignore_index=True)

#     return df


# def get_max_deadline(assignments, assignment_name, week_number):
#     matching_rows = assignments[
#         (assignments["week_number"] == week_number)
#         & (assignments["assignment_name"] == assignment_name)
#     ]

#     max_timestamp = matching_rows["due_date"].max()
#     return max_timestamp


def calculate_averages(grades_dict, student_id):
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(
        grades_dict
    ).T  # Transpose the dictionary to make assignments rows
    df["score"] = df["score"].astype(float)  # Ensure the scores are floats

    # Create a dictionary for the row data
    row_data = {"student_id": student_id}

    # Add scores for each assignment as separate columns
    for assignment, details in grades_dict.items():
        column_name = (
            f"{details['assignment_name']}_week_{details['week_number']}".replace(
                "_", " "
            )
        )
        row_data[column_name] = float(details["score"])

    # Calculate and add average scores for each assignment category
    averages = df.groupby("assignment_name")["score"].mean()
    for assignment_name, avg_score in averages.items():
        row_data[f"{assignment_name}_average".replace("_", " ")] = avg_score

    # # Course Percentage Performance
    row_data["Current Course Percentage"] = (
        (row_data["quiz average"] + row_data["practice quiz average"] * 0.1) * 15
        + (row_data["homework average"]) * 15
        + row_data["lab average"] * 15
        + row_data["lecture average"] * 15
        + row_data["readings average"] * 15
        + row_data["attendance average"] * 5
    ) / (15 + 15 + 15 + 15 + 15 + 5)

    # Create a single-row DataFrame
    single_row_df = pd.DataFrame([row_data])
    return single_row_df


def recalculate_best_grades(
    assignments,
    submissions,
    custom_order=[
        "lecture",
        "homework",
        "lab",
        "attendance",
        "readings",
        "quiz",
        "practice quiz",
    ],
):
    assignments["assignment_name"] = pd.Categorical(
        assignments["assignment_name"], categories=custom_order, ordered=True
    )

    # Sort by 'Category' (custom order) and then by 'Value'
    sorted_df = assignments.sort_values(by=["assignment_name", "week_number"])

    week_number = 3

    exclude_types = None

    sorted_df = filter_assignments(sorted_df, exclude_types=None, max_week=week_number)

    # Get unique pairs of 'week_number' and 'assignment_name'
    unique_pairs = sorted_df[["week_number", "assignment_name"]].drop_duplicates()

    # Build an iterator over the unique pairs
    pair_iterator = unique_pairs.itertuples(index=False, name=None)

    grade_dictionary = {}

    # Filter rows matching each pair
    for week_number, assignment_name in pair_iterator:
        matching_rows = sorted_df[
            (sorted_df["week_number"] == week_number)
            & (sorted_df["assignment_name"] == assignment_name)
        ]

        grade_dictionary[f"Week {week_number} {assignment_name}"] = {
            "week_number": week_number,
            "assignment_name": assignment_name,
            "score": 0,
        }

        # # extracts the names of the assignments grades which are associated with a given assignment, this helps if two assignments have the same name.
        # title = matching_rows["title"]

        # print(title)

        # gets the most recent assignment due date.
        assignment_due_date = get_max_deadline(
            assignments, assignment_name, week_number
        )
        max_score = matching_rows["max_score"].min()

        # extracts the names of the assignments grades which are associated with a given assignment, this helps if two assignments have the same name.
        for assignment_db_name in matching_rows["title"]:
            assignment_submission = submissions[
                submissions["assignment"] == assignment_db_name.replace(" ", "").lower()
            ]

            # Now we need to get the best score for an assignment based on the submission time.
            for index, submission in assignment_submission.iterrows():
                raw_score = submission["raw_score"] / max_score

                assignment_due_date = datetime.fromisoformat(
                    assignment_due_date
                ).strftime("%Y-%m-%d %H:%M:%S")
                submission_timestamp = datetime.fromisoformat(
                    submission["timestamp"]
                ).strftime("%Y-%m-%d %H:%M:%S")

                deflation_fraction = calculate_late_submission(
                    assignment_due_date, submission_timestamp
                )

                score = raw_score * deflation_fraction

                if (
                    score
                    > grade_dictionary[f"Week {week_number} {assignment_name}"]["score"]
                ):
                    grade_dictionary[f"Week {week_number} {assignment_name}"][
                        "score"
                    ] = score

    return grade_dictionary

In [ ]:
import requests
from build.passwords import password, user
from requests.auth import HTTPBasicAuth

api_base_url = "https://engr-131-api.eastus.cloudapp.azure.com/"


def get_all_students():
    res = requests.get(
        url=api_base_url.rstrip("/") + "/get-all-submission-emails",
        auth=HTTPBasicAuth(user(), password()),
    )

    return res.json()


# def get_student_grades(student_id):
#     params = {"username": student_id}
#     res = requests.get(
#         url=api_base_url.rstrip("/") + "/student-grades-testing",
#         params=params,
#         auth=HTTPBasicAuth(user(), password()),
#     )

#     [assignments, sub] = res.json()

#     return assignments, sub


def get_all_student_grades():
    # Initialize an empty DataFrame to hold all student grades
    df = pd.DataFrame()

    # Get all students (assuming get_all_students() is a defined function)
    students = get_all_students()

    for student in students:
        print(student)
        # Get assignments and submissions for the student (assumed functions)
        assignments, submissions = get_student_grades(student)

        # Recalculate grades and get a grades dictionary
        grades_dict = recalculate_best_grades(assignments, submissions)

        # Calculate averages and build a row for the student
        row = calculate_averages(grades_dict, student)

        # # Convert the row (a dictionary) into a DataFrame
        # row_df = pd.DataFrame([row])

        # Append the row to the DataFrame
        df = pd.concat([df, row], ignore_index=True)

    return df


def get_max_deadline(assignments, assignment_name, week_number):
    matching_rows = assignments[
        (assignments["week_number"] == week_number)
        & (assignments["assignment_name"] == assignment_name)
    ]

    max_timestamp = matching_rows["due_date"].max()
    return max_timestamp


def calculate_averages(grades_dict, student_id):
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(
        grades_dict
    ).T  # Transpose the dictionary to make assignments rows
    df["score"] = df["score"].astype(float)  # Ensure the scores are floats

    # Create a dictionary for the row data
    row_data = {"student_id": student_id}

    # Add scores for each assignment as separate columns
    for assignment, details in grades_dict.items():
        column_name = (
            f"{details['assignment_name']}_week_{details['week_number']}".replace(
                "_", " "
            )
        )
        row_data[column_name] = float(details["score"])

    # Calculate and add average scores for each assignment category
    averages = df.groupby("assignment_name")["score"].mean()
    for assignment_name, avg_score in averages.items():
        row_data[f"{assignment_name}_average".replace("_", " ")] = avg_score

    # # Course Percentage Performance
    row_data["Current Course Percentage"] = (
        (row_data["quiz average"] + row_data["practice quiz average"] * 0.1) * 15
        + (row_data["homework average"]) * 15
        + row_data["lab average"] * 15
        + row_data["lecture average"] * 15
        + row_data["readings average"] * 15
        + row_data["attendance average"] * 5
    ) / (15 + 15 + 15 + 15 + 15 + 5)

    # Create a single-row DataFrame
    single_row_df = pd.DataFrame([row_data])
    return single_row_df


def recalculate_best_grades(
    assignments,
    submissions,
    custom_order=[
        "lecture",
        "homework",
        "lab",
        "attendance",
        "readings",
        "quiz",
        "practice quiz",
    ],
):
    assignments["assignment_name"] = pd.Categorical(
        assignments["assignment_name"], categories=custom_order, ordered=True
    )

    # Sort by 'Category' (custom order) and then by 'Value'
    sorted_df = assignments.sort_values(by=["assignment_name", "week_number"])

    week_number = 3

    exclude_types = None

    sorted_df = filter_assignments(sorted_df, exclude_types=None, max_week=week_number)

    # Get unique pairs of 'week_number' and 'assignment_name'
    unique_pairs = sorted_df[["week_number", "assignment_name"]].drop_duplicates()

    # Build an iterator over the unique pairs
    pair_iterator = unique_pairs.itertuples(index=False, name=None)

    grade_dictionary = {}

    # Filter rows matching each pair
    for week_number, assignment_name in pair_iterator:
        matching_rows = sorted_df[
            (sorted_df["week_number"] == week_number)
            & (sorted_df["assignment_name"] == assignment_name)
        ]

        grade_dictionary[f"Week {week_number} {assignment_name}"] = {
            "week_number": week_number,
            "assignment_name": assignment_name,
            "score": 0,
        }

        # # extracts the names of the assignments grades which are associated with a given assignment, this helps if two assignments have the same name.
        # title = matching_rows["title"]

        # print(title)

        # gets the most recent assignment due date.
        assignment_due_date = get_max_deadline(
            assignments, assignment_name, week_number
        )
        max_score = matching_rows["max_score"].min()

        # extracts the names of the assignments grades which are associated with a given assignment, this helps if two assignments have the same name.
        for assignment_db_name in matching_rows["title"]:
            assignment_submission = submissions[
                submissions["assignment"] == assignment_db_name.replace(" ", "").lower()
            ]

            # Now we need to get the best score for an assignment based on the submission time.
            for index, submission in assignment_submission.iterrows():
                raw_score = submission["raw_score"] / max_score

                assignment_due_date = datetime.fromisoformat(
                    assignment_due_date
                ).strftime("%Y-%m-%d %H:%M:%S")
                submission_timestamp = datetime.fromisoformat(
                    submission["timestamp"]
                ).strftime("%Y-%m-%d %H:%M:%S")

                deflation_fraction = calculate_late_submission(
                    assignment_due_date, submission_timestamp
                )

                score = raw_score * deflation_fraction

                if (
                    score
                    > grade_dictionary[f"Week {week_number} {assignment_name}"]["score"]
                ):
                    grade_dictionary[f"Week {week_number} {assignment_name}"][
                        "score"
                    ] = score

    return grade_dictionary

In [ ]:
get_student("jab864")

In [ ]:
df = get_all_student_grades()

In [ ]:
df.to_csv("grades")